In [ ]:
import mbuild as mb
import mdtraj as md
import simtk.unit as u
import parmed as pmd
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.openmm.app.forcefield import ForceField

In [ ]:
h2o = mb.load('files/water.mol2')
h2o.name = 'HOH'

In [ ]:
system = mb.fill_box(compound=h2o,
                    n_compounds=512,
                    box=[3,3,3])

In [ ]:
system_box = mb.Box([3.5, 3.5, 3.5])
solvent = system.to_parmed(residues='HOH', box=system_box)

In [ ]:
solvent.save('init.pdb', overwrite=True)

In [ ]:
temp = 300 * u.kelvin
pressure = 1 * u.bar
timestep = 0.001 * u.picoseconds
sim_time = 1 * u.nanoseconds

In [ ]:
molecule = PDBFile('init.pdb')
amoeba = ForceField('amoeba2013.xml')
modeller = app.Modeller(molecule.topology, molecule.positions)
system = amoeba.createSystem(modeller.topology,
                            nonbondedMethod=PME,
                            nonbondedCutoff=12*u.angstroms,
                            rigidWater=False,
                            constraints=HBonds)
integrator = VerletIntegrator(timestep)

In [ ]:
sim = Simulation(modeller.topology, system, integrator)
sim.context.setPositions(modeller.positions)
sim.context.setVelocitiesToTemperature(temp)
sim.context.setPeriodicBoxVectors(
    modeller.topology.getPeriodicBoxVectors()[0],
    modeller.topology.getPeriodicBoxVectors()[1],
    modeller.topology.getPeriodicBoxVectors()[2])

In [ ]:
sim.minimizeEnergy(maxIterations=10)